<a href="https://colab.research.google.com/github/miller00315/ia_studies/blob/main/engenharia_de_atributos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler

In [4]:
dataset = pd.read_csv('credit_simple.csv', sep=';')

dataset.shape

(1000, 8)

In [5]:
dataset.head()

,SALDO_ATUAL,RESIDENCIADESDE,IDADE,OUTROSPLANOSPGTO,DATA,ESTADOCIVIL,PROPOSITO,CLASSE
0,1169.0,4,67,nenhum,01/01/2019,masculino solteiro,radio/tv,bom
1,5951.0,2,22,nenhum,01/01/2020,fem div/cas,radio/tv,ruim
2,2096.0,3,49,nenhum,02/01/2020,masculino solteiro,educação,bom
3,7882.0,4,45,nenhum,02/01/2019,masculino solteiro,mobilia/equipamento,bom
4,4870.0,4,53,nenhum,03/01/2018,masculino solteiro,carro novo,ruim


In [6]:
y = dataset['CLASSE']

X = dataset.iloc[:, :-1]

In [7]:
X.head()

,SALDO_ATUAL,RESIDENCIADESDE,IDADE,OUTROSPLANOSPGTO,DATA,ESTADOCIVIL,PROPOSITO
0,1169.0,4,67,nenhum,01/01/2019,masculino solteiro,radio/tv
1,5951.0,2,22,nenhum,01/01/2020,fem div/cas,radio/tv
2,2096.0,3,49,nenhum,02/01/2020,masculino solteiro,educação
3,7882.0,4,45,nenhum,02/01/2019,masculino solteiro,mobilia/equipamento
4,4870.0,4,53,nenhum,03/01/2018,masculino solteiro,carro novo


**Abaixo descobrimos quais valores pode aparecer nulos no datase**t

In [8]:
X.isnull().sum()

SALDO_ATUAL         7
RESIDENCIADESDE     0
IDADE               0
OUTROSPLANOSPGTO    0
DATA                0
ESTADOCIVIL         8
PROPOSITO           0
dtype: int64

In [9]:
mediana = X['SALDO_ATUAL'].median()

mediana

2323.0

**Abaixo utilizamos a mediana para preeencher os valores faltantes, a média pode causar uma discrepância devido a sensibilidade a outsiders.**

In [10]:
X['SALDO_ATUAL'].fillna(mediana, inplace=True)

X.isnull().sum()

SALDO_ATUAL         0
RESIDENCIADESDE     0
IDADE               0
OUTROSPLANOSPGTO    0
DATA                0
ESTADOCIVIL         8
PROPOSITO           0
dtype: int64

**Abaixo, para obtermos a moda utilizamos "group by"**

In [11]:
grouped = X.groupby(['ESTADOCIVIL']).size()

grouped

ESTADOCIVIL
fem div/cas               308
masculino casado/viuvo     92
masculino div/sep          50
masculino solteiro        542
dtype: int64

**Abaixo preenchemos o campo nulo com o valor moda**

In [12]:
X['ESTADOCIVIL'].fillna('masculino solteiro', inplace=True)

X.isnull().sum()

SALDO_ATUAL         0
RESIDENCIADESDE     0
IDADE               0
OUTROSPLANOSPGTO    0
DATA                0
ESTADOCIVIL         0
PROPOSITO           0
dtype: int64

**Verificando o valor do desvio**

In [13]:
desv = X['SALDO_ATUAL'].std()

desv

685936688.9820067

**Encontramos os outiliers baseados no valor do desvio padrão**

In [14]:
X.loc[X['SALDO_ATUAL'] >= 2 * desv, 'SALDO_ATUAL']

127    2.541111e+09
160    2.154441e+10
Name: SALDO_ATUAL, dtype: float64

**Encontramos a mediana**

In [15]:
mediana = X['SALDO_ATUAL'].median()

mediana

2323.0

**Atribuo o valor da mediana aos outliers**

In [16]:
X.loc[X['SALDO_ATUAL'] >= 2 * desv, 'SALDO_ATUAL'] = mediana

X.loc[X['SALDO_ATUAL'] >= 2 * desv]

,SALDO_ATUAL,RESIDENCIADESDE,IDADE,OUTROSPLANOSPGTO,DATA,ESTADOCIVIL,PROPOSITO


**Verificando valores que estão subrepresentados**

In [17]:
agrupado = X.groupby(['PROPOSITO']).size()

agrupado

PROPOSITO
Eletrodomésticos        12
carro novo             234
carro usado            103
educação                50
mobilia/equipamento    181
negócios                97
obras                   22
outros                  12
qualificação             9
radio/tv               280
dtype: int64

**Vamos unificar os valores subrepresentados por outros**

In [18]:
X.loc[X['PROPOSITO']=='Eletrodomésticos','PROPOSITO'] = 'outros'
X.loc[X['PROPOSITO']=='qualificação','PROPOSITO'] = 'outros'

agrupado = X.groupby(['PROPOSITO']).size()

agrupado

PROPOSITO
carro novo             234
carro usado            103
educação                50
mobilia/equipamento    181
negócios                97
obras                   22
outros                  33
radio/tv               280
dtype: int64

**Estração de caracteristicas.**

In [19]:
X['DATA']

0      01/01/2019
1      01/01/2020
2      02/01/2020
3      02/01/2019
4      03/01/2018
          ...    
995    29/06/2018
996    30/06/2018
997    03/07/2018
998    04/07/2019
999    05/07/2018
Name: DATA, Length: 1000, dtype: object

In [20]:
X['DATA'] = pd.to_datetime(X['DATA'], format='%d/%m/%Y')

X['DATA']

0     2019-01-01
1     2020-01-01
2     2020-01-02
3     2019-01-02
4     2018-01-03
         ...    
995   2018-06-29
996   2018-06-30
997   2018-07-03
998   2019-07-04
999   2018-07-05
Name: DATA, Length: 1000, dtype: datetime64[ns]

In [21]:
X['ANO'] = X['DATA'].dt.year

X['MES'] = X['DATA'].dt.month

X['DIASEMANA'] = X['DATA'].dt.day_name()

In [ ]:
X['DIASEMANA']

0        Tuesday
1      Wednesday
2       Thursday
3      Wednesday
4      Wednesday
         ...    
995       Friday
996     Saturday
997      Tuesday
998     Thursday
999     Thursday
Name: DIASEMANA, Length: 1000, dtype: object

**Label encoder atribui para informaçãoes categoricas valores numericos**

In [22]:
X['ESTADOCIVIL'].unique()

array(['masculino solteiro', 'fem div/cas', 'masculino div/sep',
       'masculino casado/viuvo'], dtype=object)

In [ ]:
X['PROPOSITO'].unique()

array(['radio/tv', 'educação', 'mobilia/equipamento', 'carro novo',
       'carro usado', 'negócios', 'outros', 'obras'], dtype=object)

In [23]:
X['DIASEMANA'].unique()

array(['Tuesday', 'Wednesday', 'Thursday', 'Saturday', 'Sunday', 'Monday',
       'Friday'], dtype=object)

In [24]:
labelencoder1 = LabelEncoder()

Fazendo o label encoder

In [25]:
X['ESTADOCIVIL'] = labelencoder1.fit_transform(X['ESTADOCIVIL'])

X['PROPOSITO'] = labelencoder1.fit_transform(X['PROPOSITO'])

X['DIASEMANA'] = labelencoder1.fit_transform(X['DIASEMANA'])


X.head()

,SALDO_ATUAL,RESIDENCIADESDE,IDADE,OUTROSPLANOSPGTO,DATA,ESTADOCIVIL,PROPOSITO,ANO,MES,DIASEMANA
0,1169.0,4,67,nenhum,2019-01-01,3,7,2019,1,5
1,5951.0,2,22,nenhum,2020-01-01,0,7,2020,1,6
2,2096.0,3,49,nenhum,2020-01-02,3,2,2020,1,4
3,7882.0,4,45,nenhum,2019-01-02,3,3,2019,1,6
4,4870.0,4,53,nenhum,2018-01-03,3,0,2018,1,6


**Aora vamos utilizar o one hot encoding**

In [26]:
outros = X['OUTROSPLANOSPGTO'].unique()

outros

array(['nenhum', 'banco', 'stores'], dtype=object)

**É criado um dataframe separado com os valores e as colunas com o o prefixo definido**

In [27]:
z =  pd.get_dummies(X['OUTROSPLANOSPGTO'], prefix='OUTROS')

z

,OUTROS_banco,OUTROS_nenhum,OUTROS_stores
0,0,1,0
1,0,1,0
2,0,1,0
3,0,1,0
4,0,1,0
...,...,...,...
995,0,1,0
996,0,1,0
997,0,1,0
998,0,1,0


In [28]:
X

,SALDO_ATUAL,RESIDENCIADESDE,IDADE,OUTROSPLANOSPGTO,DATA,ESTADOCIVIL,PROPOSITO,ANO,MES,DIASEMANA
0,1169.0,4,67,nenhum,2019-01-01,3,7,2019,1,5
1,5951.0,2,22,nenhum,2020-01-01,0,7,2020,1,6
2,2096.0,3,49,nenhum,2020-01-02,3,2,2020,1,4
3,7882.0,4,45,nenhum,2019-01-02,3,3,2019,1,6
4,4870.0,4,53,nenhum,2018-01-03,3,0,2018,1,6
...,...,...,...,...,...,...,...,...,...,...
995,1736.0,4,31,nenhum,2018-06-29,0,3,2018,6,0
996,3857.0,4,40,nenhum,2018-06-30,2,1,2018,6,2
997,804.0,4,38,nenhum,2018-07-03,3,7,2018,7,5
998,1845.0,4,23,nenhum,2019-07-04,3,7,2019,7,4


**Abaixo normalizamos alguns dados**

In [31]:
sc = StandardScaler()

m = sc.fit_transform(X.iloc[:, 0: 3])

m

array([[-0.74551643,  1.04698668,  1.6392759 ],
       [ 0.95774038, -0.76597727, -0.74024139],
       [-0.41533679,  0.14050471,  0.68746898],
       ...,
       [-0.87552244,  1.04698668,  0.1058092 ],
       [-0.50473818,  1.04698668, -0.68736323],
       [ 0.46799171,  1.04698668, -0.47585058]])

In [37]:
X = pd.concat([X, z, pd.DataFrame(m, columns=['SALDO_ATUAL_N', 'RESIDENCEADESDDE_N', 'IDADE_N'])], axis=1)

In [38]:
X.head()

,SALDO_ATUAL,RESIDENCIADESDE,IDADE,OUTROSPLANOSPGTO,DATA,ESTADOCIVIL,PROPOSITO,ANO,MES,DIASEMANA,OUTROS_banco,OUTROS_nenhum,OUTROS_stores,SALDO_ATUAL_N,RESIDENCEADESDDE_N,IDADE_N
0,1169.0,4,67,nenhum,2019-01-01,3,7,2019,1,5,0,1,0,-0.745516,1.046987,1.639276
1,5951.0,2,22,nenhum,2020-01-01,0,7,2020,1,6,0,1,0,0.957740,-0.765977,-0.740241
2,2096.0,3,49,nenhum,2020-01-02,3,2,2020,1,4,0,1,0,-0.415337,0.140505,0.687469
3,7882.0,4,45,nenhum,2019-01-02,3,3,2019,1,6,0,1,0,1.645526,1.046987,0.475956
4,4870.0,4,53,nenhum,2018-01-03,3,0,2018,1,6,0,1,0,0.572709,1.046987,0.898982


In [39]:
X.drop(columns=['SALDO_ATUAL', 'RESIDENCIADESDE', 'IDADE', 'OUTROSPLANOSPGTO', 'DATA', 'OUTROS_banco'], inplace=True)

In [40]:
X.head()

,ESTADOCIVIL,PROPOSITO,ANO,MES,DIASEMANA,OUTROS_nenhum,OUTROS_stores,SALDO_ATUAL_N,RESIDENCEADESDDE_N,IDADE_N
0,3,7,2019,1,5,1,0,-0.745516,1.046987,1.639276
1,0,7,2020,1,6,1,0,0.957740,-0.765977,-0.740241
2,3,2,2020,1,4,1,0,-0.415337,0.140505,0.687469
3,3,3,2019,1,6,1,0,1.645526,1.046987,0.475956
4,3,0,2018,1,6,1,0,0.572709,1.046987,0.898982
